# P-values in ML

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy as sc
import sklearn as sk
import sys
import os

from scipy.stats import chi2
from sklearn.ensemble import RandomForestClassifier
from statsmodels.discrete.discrete_model import Logit

### Create dataset

In [2]:
def logit(h):
    ''' Logistic from activation h '''
    p = 1.0 / (1.0 + np.exp(-h))
    r = np.random.rand(len(p))
    y = (r < p).astype('float')
    return y


def rand_date():
    max_time = int(time.time())
    t = random.randint(0, max_time)
    return time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(t))


def rand_unif(num, mean, std, na_prob=0):
    xi = np.random.rand(num)
    if na_prob > 0:
        xi_na = (np.random.rand(num) <= na_prob)
        xi[xi_na] = np.nan
    return xi


def rand_norm(num, mean=0.0, std=1.0, na_prob=0):
    xi = np.random.normal(mean, std, num)
    if na_prob > 0:
        xi_na = (np.random.rand(num) <= na_prob)
        xi[xi_na] = np.nan
    return xi


def rbf(x, mu=0, sigma=1):
    """ Radial basis function """
    z = (x - mu) / sigma
    return np.exp(-(z*z))


def create_dataset_01(num=1000, n_rands=5, save=False):
    """ Create dataset y = f(x1, x2, x3) + noise (r* are not used) """
    x1 = rand_norm(num)
    x2 = rand_norm(num)
    x3 = rand_norm(num)
    n = rand_norm(num)
    y = logit(3.0 * x1 - 2.0 * x2 + 1.0 * x3 + 0.5 * n)
    d = {'x1': x1, 'x2': x2, 'x3': x3, 'y': y}
    for i in range(n_rands):
        d[f"rand_{i}"] = rand_norm(num)
    df = pd.DataFrame(d)
    if save:
        file = 'zzz.csv'
        print(f"Saving dataset to file '{file}'")
        df.to_csv(file, index=False)
    return df

def create_dataset_02(num=1000, n_rands=5, save=False):
    """ Create dataset y = f(x1, x2, x3) + noise (r* are not used) """
    x1 = rand_norm(num)
    x2 = rand_norm(num)
    x3 = rand_norm(num)
    n = rand_norm(num)
    y = logit(5.0 * rbf(x1, 1, 1) - 3.0 * rbf(x2, -1, 1) + 1.0 * x3 + 0.5 * n)
    d = {'x1': x1, 'x2': x2, 'x3': x3, 'y': y}
    for i in range(n_rands):
        d[f"rand_{i}"] = rand_norm(num)
    df = pd.DataFrame(d)
    if save:
        file = 'zzz.csv'
        print(f"Saving dataset to file '{file}'")
        df.to_csv(file, index=False)
    return df

In [3]:
df = create_dataset_02(num=1000, n_rands=5, save=False)
df.head(20)

,x1,x2,x3,y,rand_0,rand_1,rand_2,rand_3,rand_4
0,0.071989,1.228345,0.513817,1.0,1.699313,0.235130,-0.472511,-0.059449,2.555528
1,-2.292641,-0.023711,0.522824,1.0,1.030342,-1.498328,-0.419560,0.675102,0.570225
2,0.238521,1.361746,-0.185919,1.0,0.210750,-0.879407,-0.296618,0.164470,-1.565242
3,0.664528,-0.089017,-1.768421,0.0,-0.363955,-0.721388,-0.390103,-0.906962,0.929828
4,2.083098,0.360569,1.510164,1.0,-0.710045,0.403853,-0.932012,-0.151162,-0.451792
5,-0.380565,-0.080786,1.460891,0.0,0.357361,-0.381817,1.996102,-0.255528,-0.235307
6,-0.566295,-0.048438,0.677421,0.0,0.358595,-0.084507,0.615222,1.317548,-0.185123
7,0.759054,1.556993,1.253496,1.0,0.290778,-0.211968,-0.819959,1.864943,1.391100
8,-0.935594,1.823721,0.818693,0.0,0.664557,0.131408,1.390804,-0.488160,-1.342137
9,-0.459485,-1.763928,0.267758,0.0,-0.463210,1.137228,0.422066,-0.350187,1.009004


In [4]:
# sns.pairplot(df, kind='scatter', diag_kind='kde')

# P-values and feature importance

In [5]:
def wilks_model_fit(x, y):
    logit_model = Logit(y, x)
    res = logit_model.fit(disp=0)
    return logit_model, res

def wilks_p_value(df, var_output, vars_null):
    model_null, model_null_results = wilks_model_fit(df[vars_null], df[var_output])
    pvalues = dict()
    for c in df.columns:
        if c != var_output and c not in vars_null:
            xnames = list(vars_null)
            xnames.append(c)
            model_alt, model_alt_res = wilks_model_fit(df[xnames], df[var_output])
            if model_alt is None:
                self._error(f"Could not fit alt model for column/s {c}")
                pval = 1.0
            else:
                d = 2.0 * (model_alt_res.llf - model_null_results.llf)
                pval = chi2.sf(d, 1)
            pvalues[c] = pval
    return pd.Series(pvalues)

def feature_importance(model, x, y, var):
    score_null = model.score(x, y)
    x_shuf = x.copy()
    x_shuf[var] = x_shuf[var].sample(frac=1).values
    score_alt = model.score(x_shuf, y)
    return (score_null - score_alt) / score_null

def feature_importance_multiple_shuffle(model, x_validate, y_validate, vars_input, num_iter=10, scores=None):
    scores = dict() if scores is None else scores
    for var in vars_input:
        delta_scores = list()
        for i in range(num_iter):
            delta_score = feature_importance(model, x_validate, y_validate, var)
            delta_scores.append(delta_score)
        delta_scores = np.array(delta_scores)
        print(f"\t{var}:\tcount: {len(delta_scores)}\tmean: {delta_scores.mean()}\tstd: {delta_scores.std()}")
        # Append values to array (if any)
        scores[var] = delta_scores if var not in scores else np.append(scores[var], delta_scores).flatten()
    return scores

def split_df(df, var_inputs, var_output, train_index, val_index):
    df_train, df_val = df.iloc[train_index], df.iloc[val_index]
    return df_train[var_inputs], df_train[var_output], df_val[var_inputs], df_val[var_output]


# One model, one data split

In [6]:
from sklearn.model_selection import KFold

# Create new dataset
def create_dataset(**kwargs): return create_dataset_02(**kwargs)

# Create new model
def new_model(): return RandomForestClassifier(100)

# Create dataset
def p_values_shuffle(num_samples=100, n_rands=5, num_cv=1, num_shuffle=10, show_plot=False, pvals_df=None):
    df = create_dataset(num=num_samples, n_rands=n_rands)
    if show_plot:
        sns.pairplot(df, kind='scatter', diag_kind='kde')
    # Variables
    var_output = 'y'
    vars_input = [c for c in df.columns if c != var_output]
    vars_null = [c for c in vars_input if c.startswith('rand_')]
    # Create several models...
    scores = dict()
    if num_cv > 1:
        cv_iter = KFold(n_splits=num_cv).split(df)
    else:
        # No cross validation, split 80% / 20%
        idx = int(0.8 * len(df))
        idx_train = range(0, idx)
        idx_val = range(idx, len(df))
        cv_iter = [(idx_train, idx_val)]
    cv_count = 0
    for train_index, val_index in cv_iter:
        cv_count += 1
        # print(f"Cross validation: {cv_count} / {num_cv}")
        # Split dataset
        x_train, y_train, x_validate, y_validate = split_df(df, vars_input, var_output, train_index, val_index)
        # Create model
        model = new_model()
        model_fit = model.fit(x_train, y_train)
        # Calculate scores (shuffle)
        scores = feature_importance_multiple_shuffle(model, x_validate, y_validate, vars_input, num_iter=num_shuffle, scores=scores)
    # Calculate p-values
    pvals_df = pd.DataFrame() if pvals_df is None else pvals_df
    null_scores = np.array([scores[c] for c in vars_null]).flatten()
    for c in vars_input:
        null_scores = np.array([scores[cn] for cn in vars_null if cn != c]).flatten()
        pval = sc.stats.mannwhitneyu(scores[c], null_scores, alternative='greater')[1]
        df_row = pd.DataFrame({'name': c,
            'num_samples': num_samples, 'n_rands': n_rands,
            'num_cv': num_cv, 'num_shuffle': num_shuffle,
            'count_alt': len(scores[c])
                      , 'mean_alt': scores[c].mean(), 'std_alt': scores[c].std()
                      , 'count_null': len(null_scores), 'mean_null': null_scores.mean()
                      , 'std_null': null_scores.std()
                      , 'p_value': pval}, index=[len(pvals_df)])
        pvals_df = pvals_df.append(df_row)
    return pvals_df

In [7]:
pvals_df = None
for num_cv in [1, 3, 5, 10, 20]:
    for num_samples in [50, 100, 200, 300, 400, 500, 1000, 2000, 10000]:
        for num_shuffle in [3, 5, 10, 20, 50, 100]:
            pvals_df = p_values_shuffle(num_samples=num_samples, n_rands=5, num_cv=num_cv, num_shuffle=num_shuffle, pvals_df=pvals_df)
            print(f"num_cv: {num_cv}\tnum_samples:{num_samples}\tnum_shuffle:{num_shuffle}")

pvals_df.to_csv('p_values_shuffle.csv')

	x1:	count: 3	mean: 0.0	std: 0.0
	x2:	count: 3	mean: 0.04761904761904761	std: 0.06734350297014736
	x3:	count: 3	mean: -0.09523809523809533	std: 0.06734350297014745
	rand_0:	count: 3	mean: 0.09523809523809522	std: 0.06734350297014736
	rand_1:	count: 3	mean: 0.09523809523809522	std: 0.06734350297014736
	rand_2:	count: 3	mean: 0.0	std: 0.0
	rand_3:	count: 3	mean: 0.0	std: 0.0
	rand_4:	count: 3	mean: -0.047619047619047665	std: 0.06734350297014743
num_cv: 1	num_samples:50	num_shuffle:3
	x1:	count: 5	mean: 0.5142857142857142	std: 0.14568627181693675
	x2:	count: 5	mean: -0.02857142857142863	std: 0.1069044967649698
	x3:	count: 5	mean: -0.0571428571428572	std: 0.06998542122237658
	rand_0:	count: 5	mean: 0.0	std: 0.0
	rand_1:	count: 5	mean: -0.08571428571428578	std: 0.06998542122237658
	rand_2:	count: 5	mean: -0.02857142857142863	std: 0.1069044967649698
	rand_3:	count: 5	mean: -0.08571428571428578	std: 0.06998542122237658
	rand_4:	count: 5	mean: -0.08571428571428583	std: 0.11428571428571432
num_

	rand_3:	count: 10	mean: 0.054285714285714284	std: 0.029829447168315832
	rand_4:	count: 10	mean: 0.022857142857142878	std: 0.02138089935299397
num_cv: 1	num_samples:200	num_shuffle:10
	x1:	count: 20	mean: 0.225	std: 0.06656241849238619
	x2:	count: 20	mean: -0.011666666666666665	std: 0.06171169527191209
	x3:	count: 20	mean: 0.01833333333333334	std: 0.032446537223219656
	rand_0:	count: 20	mean: -6.938893903907229e-19	std: 0.02981423969999722
	rand_1:	count: 20	mean: 0.03166666666666667	std: 0.04530759073022729
	rand_2:	count: 20	mean: -0.006666666666666672	std: 0.02260776661041758
	rand_3:	count: 20	mean: -0.015000000000000013	std: 0.022298480267099442
	rand_4:	count: 20	mean: -0.03333333333333333	std: 0.034960294939005036
num_cv: 1	num_samples:200	num_shuffle:20
	x1:	count: 50	mean: 0.09846153846153846	std: 0.09139290301710243
	x2:	count: 50	mean: 0.03230769230769233	std: 0.05404359406230862
	x3:	count: 50	mean: -0.10461538461538454	std: 0.05758451468373199
	rand_0:	count: 50	mean: -0.0

	rand_1:	count: 50	mean: 0.01030303030303026	std: 0.015873698004495493
	rand_2:	count: 50	mean: -0.001818181818181861	std: 0.019787669960381223
	rand_3:	count: 50	mean: -0.0030303030303030815	std: 0.020774710910306193
	rand_4:	count: 50	mean: 0.003939393939393899	std: 0.013142574813455411
num_cv: 1	num_samples:400	num_shuffle:50
	x1:	count: 100	mean: 0.12716666666666665	std: 0.048913472354534165
	x2:	count: 100	mean: 0.06566666666666665	std: 0.037030017853387845
	x3:	count: 100	mean: 0.05183333333333333	std: 0.033407833412739195
	rand_0:	count: 100	mean: -0.000666666666666663	std: 0.0194250697124446
	rand_1:	count: 100	mean: 0.01683333333333332	std: 0.020614854407010056
	rand_2:	count: 100	mean: -0.007999999999999991	std: 0.02153550040695078
	rand_3:	count: 100	mean: -0.00716666666666667	std: 0.027411980349231722
	rand_4:	count: 100	mean: -0.01266666666666665	std: 0.018427033281446997
num_cv: 1	num_samples:400	num_shuffle:100
	x1:	count: 3	mean: 0.16190476190476186	std: 0.0269374011880

	rand_0:	count: 3	mean: 0.004219409282700425	std: 0.014917864581994685
	rand_1:	count: 3	mean: 0.011603375527426192	std: 0.005378712567080991
	rand_2:	count: 3	mean: 0.02215189873417726	std: 0.0025838499396447054
	rand_3:	count: 3	mean: 0.014767932489451538	std: 0.0014917864581995024
	rand_4:	count: 3	mean: 0.0253164556962026	std: 0.00931620344549355
num_cv: 1	num_samples:2000	num_shuffle:3
	x1:	count: 5	mean: 0.24827586206896549	std: 0.009171623096130302
	x2:	count: 5	mean: 0.07899686520376174	std: 0.0030714604925181107
	x3:	count: 5	mean: 0.050156739811912245	std: 0.017284073670332533
	rand_0:	count: 5	mean: 0.010031347962382397	std: 0.006981522712012579
	rand_1:	count: 5	mean: -0.0068965517241379795	std: 0.007780359652658865
	rand_2:	count: 5	mean: 0.0037617554858933927	std: 0.005393307377456217
	rand_3:	count: 5	mean: 0.015047021943573652	std: 0.005393307377456178
	rand_4:	count: 5	mean: 0.010031347962382425	std: 0.0036557692130691375
num_cv: 1	num_samples:2000	num_shuffle:5
	x1:	c

	rand_1:	count: 3	mean: 0.0	std: 0.08164965809277261
	rand_2:	count: 3	mean: -0.03333333333333333	std: 0.047140452079103175
	rand_3:	count: 3	mean: -0.06666666666666667	std: 0.04714045207910317
	rand_4:	count: 3	mean: -0.03333333333333333	std: 0.12472191289246473
num_cv: 3	num_samples:50	num_shuffle:3
	x1:	count: 5	mean: 0.1272727272727273	std: 0.1090909090909091
	x2:	count: 5	mean: 0.07272727272727274	std: 0.10601730717900548
	x3:	count: 5	mean: -0.03636363636363637	std: 0.04453617714151233
	rand_0:	count: 5	mean: 0.0	std: 0.0
	rand_1:	count: 5	mean: -0.07272727272727267	std: 0.0890723542830246
	rand_2:	count: 5	mean: 0.0	std: 0.05749595745760691
	rand_3:	count: 5	mean: -0.05454545454545455	std: 0.04453617714151233
	rand_4:	count: 5	mean: -0.07272727272727274	std: 0.03636363636363637
	x1:	count: 5	mean: 0.21538461538461534	std: 0.0575639597965222
	x2:	count: 5	mean: 0.061538461538461445	std: 0.030769230769230722
	x3:	count: 5	mean: 0.12307692307692294	std: 0.06153846153846156
	rand_0:

...

	x1:	count: 10	mean: 0.2575903614457832	std: 0.01785247849830504
	x2:	count: 10	mean: 0.08313253012048187	std: 0.01446786800219575
	x3:	count: 10	mean: 0.08554216867469874	std: 0.01329675148079172
	rand_0:	count: 10	mean: 0.0016867469879518083	std: 0.007699539907792999
	rand_1:	count: 10	mean: 0.003614457831325305	std: 0.007168421581033028
	rand_2:	count: 10	mean: 0.005060240963855426	std: 0.007423094843734639
	rand_3:	count: 10	mean: 0.010843373493975888	std: 0.008570322527122257
	rand_4:	count: 10	mean: 0.00024096385542168706	std: 0.00693791809584963
	x1:	count: 10	mean: 0.23002544529262092	std: 0.02215932932018042
	x2:	count: 10	mean: 0.08320610687022909	std: 0.017849831966713078
	x3:	count: 10	mean: 0.06666666666666672	std: 0.005189841744114798
	rand_0:	count: 10	mean: -0.0017811704834605618	std: 0.006133318469818809
	rand_1:	count: 10	mean: -0.0017811704834605614	std: 0.008366046932709217
	rand_2:	count: 10	mean: -0.004071246819338426	std: 0.004436538364926898
	rand_3:	count: 10	m

	rand_2:	count: 20	mean: -0.0014138817480719805	std: 0.009566125751331433
	rand_3:	count: 20	mean: -0.009897172236503864	std: 0.005933516137575219
	rand_4:	count: 20	mean: -0.001542416452442161	std: 0.005355441299742848
	x1:	count: 20	mean: 0.20051546391752578	std: 0.028544323492702982
	x2:	count: 20	mean: 0.0762886597938145	std: 0.021337342304968268
	x3:	count: 20	mean: 0.06829896907216501	std: 0.017375416279054836
	rand_0:	count: 20	mean: -0.006314432989690727	std: 0.00792185200479572
	rand_1:	count: 20	mean: -0.011211340206185577	std: 0.006379842162106481
	rand_2:	count: 20	mean: -0.003994845360824745	std: 0.007219945787541985
	rand_3:	count: 20	mean: 0.007087628865979387	std: 0.006909668360028104
	rand_4:	count: 20	mean: 0.000644329896907217	std: 0.007464221885347729
	x1:	count: 20	mean: 0.267206982543641	std: 0.018210892871445207
	x2:	count: 20	mean: 0.10773067331670834	std: 0.01878783860532407
	x3:	count: 20	mean: 0.0659600997506235	std: 0.014161317278822399
	rand_0:	count: 20	me

	x2:	count: 50	mean: 0.07579487179487186	std: 0.013055890577205806
	x3:	count: 50	mean: 0.08964102564102569	std: 0.018079814280521404
	rand_0:	count: 50	mean: -0.005128205128205133	std: 0.009116096838272405
	rand_1:	count: 50	mean: -0.0026666666666666687	std: 0.008268346205918915
	rand_2:	count: 50	mean: 0.0062564102564102615	std: 0.006570509693557735
	rand_3:	count: 50	mean: 0.008205128205128212	std: 0.008010512488109396
	rand_4:	count: 50	mean: -0.004871794871794876	std: 0.008615537239185362
	x1:	count: 50	mean: 0.23837037037037045	std: 0.018372826067114052
	x2:	count: 50	mean: 0.08172839506172845	std: 0.013934766478540715
	x3:	count: 50	mean: 0.07293827160493833	std: 0.01572674759284668
	rand_0:	count: 50	mean: -0.0021728395061727753	std: 0.007972517953821407
	rand_1:	count: 50	mean: -0.0016296296296295814	std: 0.006525435910658503
	rand_2:	count: 50	mean: -4.938271604934709e-05	std: 0.006761826560191913
	rand_3:	count: 50	mean: -0.0014814814814814278	std: 0.007456626602736495
	rand

	rand_3:	count: 50	mean: 0.0024578313253012076	std: 0.00548403243059442
	rand_4:	count: 50	mean: 0.021445783132530066	std: 0.007547209524513603
	x1:	count: 50	mean: 0.2330026109660575	std: 0.02680820127494379
	x2:	count: 50	mean: 0.09550913838120108	std: 0.018573422291675058
	x3:	count: 50	mean: 0.06710182767624025	std: 0.018735387154997653
	rand_0:	count: 50	mean: -0.012271540469973901	std: 0.008850366191476923
	rand_1:	count: 50	mean: -0.01603133159268931	std: 0.006828761976286472
	rand_2:	count: 50	mean: -0.014569190600522206	std: 0.00654493731727387
	rand_3:	count: 50	mean: 0.00469973890339426	std: 0.007727753831304833
	rand_4:	count: 50	mean: -0.013733681462141005	std: 0.008220356180746116
num_cv: 20	num_samples:10000	num_shuffle:50
	x1:	count: 100	mean: 0.20616966580976867	std: 0.0237757935830784
	x2:	count: 100	mean: 0.0875321336760926	std: 0.018236986534249492
	x3:	count: 100	mean: 0.05264781491002575	std: 0.016514503768773135
	rand_0:	count: 100	mean: -0.010231362467866333	std

	x2:	count: 100	mean: 0.09965260545905714	std: 0.017594154525582262
	x3:	count: 100	mean: 0.0812655086848636	std: 0.01588622351616106
	rand_0:	count: 100	mean: 0.004714640198511176	std: 0.008411142062867631
	rand_1:	count: 100	mean: -0.0007444168734491144	std: 0.007684299040795028
	rand_2:	count: 100	mean: -0.003548387096774164	std: 0.007738236375299002
	rand_3:	count: 100	mean: -0.0027295285359801224	std: 0.008075007491450577
	rand_4:	count: 100	mean: -0.00709677419354834	std: 0.007583639700460688
	x1:	count: 100	mean: 0.21469072164948458	std: 0.02538496253039947
	x2:	count: 100	mean: 0.07961340206185573	std: 0.020472728503850932
	x3:	count: 100	mean: 0.06639175257731963	std: 0.018232127085780727
	rand_0:	count: 100	mean: 0.002963917525773198	std: 0.0066049314581131495
	rand_1:	count: 100	mean: 0.004536082474226808	std: 0.00839209452430297
	rand_2:	count: 100	mean: 0.008891752577319594	std: 0.008274560699318898
	rand_3:	count: 100	mean: -0.0007731958762886605	std: 0.00872109500808693


In [8]:
pvals_df.head(20)

,name,num_samples,n_rands,num_cv,num_shuffle,count_alt,mean_alt,std_alt,count_null,mean_null,std_null,p_value
0,x1,50,5,1,3,3,0.000000,0.000000,15,0.028571,0.077372,0.775154
1,x2,50,5,1,3,3,0.047619,0.067344,15,0.028571,0.077372,0.387268
2,x3,50,5,1,3,3,-0.095238,0.067344,15,0.028571,0.077372,0.985239
3,rand_0,50,5,1,3,3,0.095238,0.067344,12,0.011905,0.070430,0.058593
4,rand_1,50,5,1,3,3,0.095238,0.067344,12,0.011905,0.070430,0.058593
5,rand_2,50,5,1,3,3,0.000000,0.000000,12,0.035714,0.085017,0.807956
6,rand_3,50,5,1,3,3,0.000000,0.000000,12,0.035714,0.085017,0.807956
7,rand_4,50,5,1,3,3,-0.047619,0.067344,12,0.047619,0.067344,0.966211
8,x1,50,5,1,5,5,0.514286,0.145686,25,-0.057143,0.090351,0.000124
9,x2,50,5,1,5,5,-0.028571,0.106904,25,-0.057143,0.090351,0.311162


In [9]:
pvals_df.tail(20)

,name,num_samples,n_rands,num_cv,num_shuffle,count_alt,mean_alt,std_alt,count_null,mean_null,std_null,p_value
2140,rand_1,10000,5,20,20,400,0.000794,0.010922,1600,0.002215,0.011567,9.542859e-01
2141,rand_2,10000,5,20,20,400,0.003337,0.011141,1600,0.001579,0.011505,3.780746e-04
2142,rand_3,10000,5,20,20,400,0.002109,0.010594,1600,0.001886,0.011660,2.928666e-01
2143,rand_4,10000,5,20,20,400,0.001525,0.011522,1600,0.002032,0.011436,7.681126e-01
2144,x1,10000,5,20,50,1000,0.232597,0.025826,5000,0.000265,0.012317,0.000000e+00
2145,x2,10000,5,20,50,1000,0.090441,0.025795,5000,0.000265,0.012317,0.000000e+00
2146,x3,10000,5,20,50,1000,0.080458,0.024175,5000,0.000265,0.012317,0.000000e+00
2147,rand_0,10000,5,20,50,1000,0.000338,0.013352,4000,0.000247,0.012044,2.665263e-01
2148,rand_1,10000,5,20,50,1000,-0.002443,0.010913,4000,0.000942,0.012553,1.000000e+00
2149,rand_2,10000,5,20,50,1000,0.000660,0.011744,4000,0.000166,0.012454,1.570716e-01
